In [6]:
system_prompt = """You are an expert in Human-Computer Interaction and user experience analysis. Your task is to reverse-engineer a user's immediate intention by analyzing a single interaction on a website.

You will be provided with the 'before' and 'after' states of a webpage, represented as simplified Accessibility (A11y) trees. You will also be given the specific action (e.g., click, scroll, type) that caused the transition from the 'before' state to the 'after' state.

Your analysis process MUST follow these two steps:
1.  **Describe the Pages:** First, for both the 'before' and 'after' A11y trees, write a concise description of the page's main purpose and key interactive elements.
2.  **Infer the Intention:** Second, based on the action and the resulting change between the two pages, infer the user's most likely, immediate intention and the resulting page. To do this, consider the following categories of user motivation:
    *   **Navigation:** Was the user trying to move to a completely new section or page with a different purpose? (e.g., clicking a 'Profile' link to go to their profile page).
    *   **Information Seeking/Exposure:** Was the user trying to reveal, expand, or access new information on the *same logical page*? (e.g., clicking a 'Show Details' button to expand a product description, hovering over an icon to see a tooltip).
    *   **State Change/Interaction:** Was the user trying to change the state of a component to prepare for a future action? (e.g., typing in a search bar, clicking a checkbox to apply a filter, selecting an option from a dropdown menu).

Your final output MUST be a single, valid JSON object containing exactly three keys: `web_desc_before`, `web_desc_after`, and `intention`. The `intention` should be a concise, action-oriented phrase that clearly describes the user's goal.

Do not add any explanations or text outside of the final JSON object."""


example_user = """##########before_page_a11y##########:
Tab 0 (current): Postmill

[1] RootWebArea 'Postmill' focused: True
	[32] HeaderAsNonLandmark ''
		[33] link 'Home'
	[65] link 'Forums'
	[66] link 'Wiki'
	[74] searchbox 'Search query'
	[75] link 'Notifications (0)'
	[76] link 'Submit'
	[12] button 'MarvelsGrantMan136' hasPopup: menu expanded: False
	[14] main ''
		[181] link 'Submissions'
		[182] link 'Comments'
		[18] button 'Filter on: Featured' hasPopup: menu expanded: False
		[20] button 'Sort by: Hot' hasPopup: menu expanded: False
		[55] StaticText 'You are not subscribed to any forum. Showing featured forums instead.'

##########after_page_a11y##########:
Tab 0 (current): Search

[243] RootWebArea 'Search' focused: True
	[256] main ''
			[1595] StaticText 'explore'
		[615] heading '50 results for explore:'
			[260] HeaderAsNonLandmark ''
				[673] heading 'I get paid to explore abandoned places. (Urban Explore) I travel, photograph, explore, and film documentaries of them. I have found a lot of cool things, I have ran into a lot of interesting people, have almost been attacked multiple times, and have seen some spooky stuff. Ask me anything.'
					[1596] link 'I get paid to explore abandoned places. (Urban Explore) I travel, photograph, explore, and film documentaries of them. I have found a lot of cool things, I have ran into a lot of interesting people, have almost been attacked multiple times, and have seen some spooky stuff. Ask me anything.'
							[1606] StaticText 'explore'
						[1603] StaticText ', and film documentaries of them. I have found a lot of cool things, I have ran into a lot of interesting people, have almost been attacked multiple times, and have seen some spooky stuff. Ask me anything.'
				[1607] StaticText 'Submitted by '
				[263] link 'Blake4582' expanded: False
				[2778] StaticText 't3_10kn86u'
				[1611] time 'January 24, 2023 at 9:08:09 PM EST'
					[2782] StaticText '3 years ago'
				[1612] StaticText ' in '
				[1613] link 'IAmA'
			[676] StaticText 'best/craziest/spooky encounters, how I got started, my favorite cities, countries, or locations, photography advice, YouTube advice, urban '
			[677] mark ''
				[1614] StaticText 'exploring'
			[678] StaticText ' advice, or anything else you might be wondering. I have amassed a respectable amount of followers'
			[2785] link 'No comments'
			[682] button 'Upvote'
			[685] button 'Downvote'

##########action##########:
type [74] [explore ]"""


example_assistant = """{
  "web_desc_before": "The main homepage of a forum website, showing primary navigation links like 'Home' and 'Forums', a user profile button, and a prominent search box.",
  "web_desc_after": "A search results page displaying a list of posts and comments that match the keyword 'explore', along with details for each result like the author and submission time.",
  "intention": "The user wants to find posts or forums related to the topic of 'exploration'. This action leads to a search results page. It presents a list of forum submissions that match the keyword 'explore', including its title, the author's username, the community it was posted in, and interactive elements like upvote and downvote buttons."
}"""

user_prompt_template = """##########before_page_a11y##########:
{before_a11y}

##########after_page_a11y##########:
{after_a11y}

##########action##########:
{action}"""



In [7]:
import csv
import random
import time
import json
import copy


import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = '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'
os.environ['LANGSMITH_PROJECT'] = 'pr-wooden-hedgehog-65'
os.environ['OPENAI_API_KEY'] = 'sk-g4MycviIFf8Lad0v5zVKPljDpueIDBLuqIC1nsDAiRnLZVKg'
os.environ['OPENAI_API_BASE'] = 'https://api2.aigcbest.top/v1'

from typing import Any, List, Dict

# LangChain 和 LangSmith 的相关导入
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ChatMessage

def generate_from_openai_chat_completion_new(
    messages: List[Dict[str, str]],
    model: str,
    temperature: float,
    max_tokens: int,
    top_p: float,
    context_length: int, # 这个参数在ChatOpenAI中不直接使用，但为了保持签名一致而保留
    stop_token: str | None = None,
) -> str:
    """
    使用 LangChain 的 ChatOpenAI 生成聊天回复，并支持 LangSmith 追踪。
    函数签名与旧版完全兼容。
    """
    if "OPENAI_API_KEY" not in os.environ and "api_key" not in locals():
        raise ValueError("OPENAI_API_KEY environment variable must be set.")

    # 1. 初始化 LangChain 的 ChatOpenAI 客户端
    # 将旧函数的参数映射到 ChatOpenAI 的构造函数中
    llm = ChatOpenAI(
        # 从环境变量获取 key 和 base_url，提供默认值
        api_key=os.environ.get("OPENAI_API_KEY"),
        base_url=os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1"),
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        stop=[stop_token] if stop_token else None,
        model_kwargs={}, # 如果没有其他高级参数，可以为空字典或直接省略
        # 内置重试机制，替代旧的装饰器
        max_retries=3,
    )

    # 2. 将输入的字典列表转换为 LangChain 的消息对象
    # 这是一个必要的适配步骤
    langchain_messages = []
    for msg in messages:
        role = msg.get("role")
        content = msg.get("content", "")
        # 从字典中获取 name，如果不存在则为 None
        name = msg.get("name")

        if role == "user":
            langchain_messages.append(HumanMessage(content=content, name=name))
        elif role == "assistant":
            langchain_messages.append(AIMessage(content=content, name=name))
        elif role == "system":
            # 这会正确处理您提供的 few-shot 示例
            langchain_messages.append(SystemMessage(content=content, name=name))
        else:
            # 为其他可能的角色（如 'tool'）提供一个通用处理
            langchain_messages.append(ChatMessage(role=role, content=content))

    # 3. 调用 LLM 并获取结果
    response = llm.invoke(langchain_messages)

    # 4. 提取并返回内容，保持与旧函数相同的字符串输出
    return response.content

In [8]:
# example_messages_with_name = [{
#     'role': 'user',
#     'content': "hello"
# }]

# result = generate_from_openai_chat_completion_new(
#     messages=example_messages_with_name,
#     model="gpt-4o",
#     temperature=0.7,
#     max_tokens=2000,
#     top_p=1.0,
#     context_length=4096,
#     stop_token=None
# )
# result

In [9]:
import re

def get_a11y_and_action_from_step(step):
    human_message_lines = step['conversations'][0]['value'].split("\n")
    a_11y_lines = []
    a_11y_start_flg = False
    for line in human_message_lines:
        if line.startswith("Accessibility tree:"):
            a_11y_start_flg = True
        elif line.startswith("Action History:"):
            a_11y_start_flg = False

        if a_11y_start_flg:
            a_11y_lines.append(line)

    llm_message = step['conversations'][1]['value']
    try:
        llm_message = re.search(r'```(.*)```', llm_message, re.DOTALL).group(1)
    except Exception as e:
        print(f"error {step['conversations'][1]['value']}")
        llm_message = "WARNING: action is empty"
    return "\n".join(a_11y_lines), llm_message

In [11]:
import json
from tqdm import tqdm

folder_path = "/home/zjusst/qms/osgensis/len_5_trajs"

orginal_file_names = os.listdir(folder_path)
# 如果已经有对应的 _reverse_intention.json 文件，则跳过原文件
file_names = []
for file_name in orginal_file_names:
    if os.path.exists(os.path.join(folder_path, f"{file_name[:-5]}_reverse_intention.json")):
        print(f"skip {file_name}")
        continue
    if file_name.endswith("intention.json"):
        print(f"skip {file_name}")
        continue
    file_names.append(file_name)

# len(file_names)


for file_name in tqdm(file_names):
    total_results = []
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, "r") as f:
        traj = json.load(f)

    for i in range(len(traj)-1):
        try:
            before_a11y, action = get_a11y_and_action_from_step(traj[i])
            after_a11y, _ = get_a11y_and_action_from_step(traj[i+1])
        except Exception as e:
            print(f"error {file_name} {i}")
            raise e

        user_prompt = user_prompt_template.format(before_a11y=before_a11y, after_a11y=after_a11y, action=action)

        example_messages_with_name = [
            {
                'role': 'system',
                'content': system_prompt
            },
            {
                'role': 'system',
                'name': 'example_user',
                'content': example_user
            },
            {
                'role': 'system',
                'name': 'example_assistant',
                'content': example_assistant
            },
            {
                'role': 'user',
                'name': 'user',
                'content': user_prompt
            }
        ]

        result = generate_from_openai_chat_completion_new(
            messages=example_messages_with_name,
            model="gpt-4o",
            temperature=0.7,
            max_tokens=2000,
            top_p=1.0,
            context_length=4096,
            stop_token=None
        )

        json_result = json.loads(result)
        total_results.append(json_result)

    # total_results 保存为 "/home/zjusst/qms/osgensis/len_2_trajs/traj_0_reverse_intention.json"
    save_path = os.path.join(folder_path, f"{file_name[:-5]}_reverse_intention.json")
    with open(save_path, "w") as f:
        json.dump(total_results, f)


  0%|          | 0/25 [00:00<?, ?it/s]

  4%|▍         | 1/25 [00:13<05:29, 13.74s/it]

error 


 48%|████▊     | 12/25 [02:20<02:29, 11.49s/it]

error 


 72%|███████▏  | 18/25 [03:37<01:23, 11.99s/it]

error 


 88%|████████▊ | 22/25 [04:35<00:41, 13.90s/it]

error 


100%|██████████| 25/25 [05:14<00:00, 12.56s/it]
